::::
:::{thebe-button}
:::
::::

# Find objects

Find bubble size using two different approaches.


In [ ]:
from devtools import pprint
from geopandas import GeoDataFrame, points_from_xy
from numpy import pi, sqrt
from pandas import DataFrame, IndexSlice, NamedAgg
from seaborn import scatterplot
from shapely import LinearRing, Polygon

from boilercv.data import FRAME
from boilercv.images import scale_bool
from boilercv_docs.nbs import init
from boilercv_pipeline.models.deps import get_slices
from boilercv_pipeline.sets import get_contours_df2, get_dataset2
from boilercv_pipeline.stages.common.e230920 import GBC, bounded_ax
from boilercv_pipeline.stages.common.e230920.types import DfNbOuts
from boilercv_pipeline.stages.e230920_find_objects import Deps, get_nb
from boilercv_pipeline.stages.e230920_find_objects import E230920FindObjects as Params

PARAMS = None

In [ ]:
if isinstance(PARAMS, str):
    params = Params.model_validate_json(PARAMS)
else:
    params = Params(
        context=(ctx := init()),
        deps=(deps := Deps(context=ctx)),
        compare_with_trackpy=True,
        nb=get_nb(deps),
    )
pprint(params)

In [ ]:
contours_df = get_contours_df2(params.nb.contours)
slices = get_slices(params.nb.filled_slicers)
frames = slices.get(FRAME, slice(None))
filled = scale_bool(get_dataset2(params.nb.filled, slices=slices)["video"])
params.format.set_display_options()

## Data

Load a video of filled contours and the contour loci and plot a composite of all frames to analyze.


In [ ]:
if params.compare_with_trackpy:
    from trackpy import batch

composite_video = filled.max("frame").values
with bounded_ax(composite_video) as ax:
    ax.imshow(~composite_video, alpha=0.4)

params.hide()

## Find size from filled contours using Trackpy

Use Trackpy to find bubble size given the filled contours.


In [ ]:
if params.compare_with_trackpy:
    tp_objects = (
        batch(
            frames=filled.values, diameter=params.guess_diameter, characterize=True
        ).assign(
            frame=lambda df: df.frame.replace(dict(enumerate(filled.frame.values)))
        )
    ).loc[:, params.trackpy_cols]
else:
    tp_objects = DataFrame()

tp_objects

## Find size from contours

The prior approach throws out contour data, instead operating on filled contours. Instead, try using shapely to find size directly from contour data.

### Prepare to find objects

Prepare a dataframe with columns in a certain order, assign contour data to it, and demote the hiearchical indices to plain columns. Count the number of points in each contour and each frame, keeping only those which have enough points to describe a linear ring. Construct a GeoPandas geometry column and operate on it with Shapely to construct linear rings, returning centroids and the representative polygonal area. Also report the number of points in the loci of each contour per frame.


In [ ]:
contours = (
    DataFrame(columns=["xpx", "ypx"])
    .assign(**contours_df.loc[IndexSlice[frames, :], :])
    .rename(axis="columns", mapper=dict(xpx="x", ypx="y"))
    .reset_index()
    .assign(
        count=lambda df: df.groupby(["frame", "contour"], **GBC).x.transform("count")
    )
    .query("count > 3")
    .assign(geometry=lambda df: points_from_xy(df.x, df.y))
    .groupby(["frame", "contour"], **GBC)
    .agg(
        count=NamedAgg(column="count", aggfunc="first"),
        centroid=NamedAgg(
            column="geometry", aggfunc=lambda df: LinearRing(df).centroid
        ),
        area=NamedAgg(column="geometry", aggfunc=lambda df: Polygon(df).area),
    )
    .assign(
        diameter_px=lambda df: sqrt(4 * df["area"] / pi),
        radius_of_gyration_px=lambda df: df["diameter_px"] / 4,
        size=lambda df: df["radius_of_gyration_px"],
    )
)
contours

Split the centroid point objects into separate named columns that conform to the Trackpy convention. Report the centroids in each frame.


In [ ]:
outs = DfNbOuts(
    df=(
        GeoDataFrame(contours)
        .assign(x=lambda df: df.centroid.x, y=lambda df: df.centroid.y)
        .loc[:, params.cols]
        .sort_values(["frame", "y", "x"], ignore_index=True)
    )
)
outs.df

## Compare approaches

Compare Trackpy objects with contour objects. Here the guess radius for Trackpy object finding and contour perimeter filtering are matched to produce the same number of objects from each algorithm. Trackpy features more intelligent filtering, but takes much longer. Trackpy's approach for finding local maxima in grayscale images is applied even to binarized images, exhaustively searching for high points in the binary image, adding to execution time.

The percent difference between the approaches is relatively low for this subset, suggesting the contour centroid approach is reasonable.

A warm color palette is used to plot Trackpy objects, and a cool color palette is used to plot contour centroids.


In [ ]:
if params.compare_with_trackpy:
    scatterplot(
        ax=ax, data=tp_objects, x="x", y="y", alpha=0.6, color="red", legend=False
    )
scatterplot(ax=ax, data=outs.df, x="x", y="y", alpha=0.6, color="blue", legend=False)

fig = ax.get_figure()
fig